In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import keras.utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.regularizers as kr
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.losses as ls
import matplotlib.pyplot as plt
import sys
import pydot
np.set_printoptions(threshold=sys.maxsize)
%matplotlib qt

In [ ]:
###Vengono caricati i dati dal file, questa volta non viene fatta nessuna selezione a priori
###a parte quella per numero di elementi non nulli.

data=pd.read_csv('./Data/DEFMSData.csv')
nancount=[]
ind=[]
for i in range(0,data.shape[0]):
    nancount.append(data.iloc[i,12:].isnull().sum())
    if nancount[i]<5:
        ind.append(i)
    else:
        pass
data=data.iloc[ind,:]
data=data.set_index(['GES_FLD'])

data=data.reset_index()   
dataset=data.values

invar=dataset[:,12:].astype(float)
invar=np.nan_to_num(invar)


In [ ]:
###Nuova NN. Vengono caricati i files preparati con il training e associati ai rispettivi layers. NB Di default considera i
###weights trovati da me. Per utilizzare glil eventuali nuovi weights, sostituire qui sotto FixedWeights con EvalWeights.

model=Sequential()
model.add(Dense(invar.shape[1],input_dim=invar.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(round(invar.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(3,activation='softsign'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
w0=np.load("./FixedWeights/msl0.npy")
b0=np.load("./FixedWeights/msl0bias.npy")
w1=np.load("./FixedWeights/msl1.npy")
b1=np.load("./FixedWeights/msl1bias.npy")
w2=np.load("./FixedWeights/msl2.npy")
b2=np.load("./FixedWeights/msl2bias.npy")
model.layers[0].set_weights([w0,b0])
model.layers[1].set_weights([w1,b1])
model.layers[2].set_weights([w2,b2])

In [ ]:
###Viene fatta una run con tutte le stelle, si selezionano quali visualizzare direttamente nei grafici.

out=model.predict(invar)

In [ ]:
###Primo grafico con OC nuovi (NON QUELLI DEL TRAINING)

import matplotlib.gridspec as gridspec
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure()
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('n_(0)')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('n_(2)')
ax2.set_xlabel('n_(1)')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('n_(2)')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('n_(1)')
ax3.set_ylabel('n_(0)')
ax3.yaxis.set_label_position('right')
ax1.scatter(out[340,0],out[340,2],color='red',marker='+',label='Pleiades')
ax1.scatter(out[130,0],out[130,2],color='blue',marker='1',label='NGC2264')
ax1.scatter(out[50,0],out[50,2],color='green',marker='4',label='IC2602')
ax1.scatter(out[110,0],out[110,2],color='cyan',marker='x',label='NGC2232')
ax1.scatter(out[70,0],out[70,2],color='black',marker='.',label='IC4665')

for i in range(0,dataset.shape[0]):
    if dataset[i,0]=='Pleiades':
        ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif dataset[i,0]=='NGC2264':
        ax1.scatter(out[i,0],out[i,2],color='blue',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='blue',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='blue',marker='1')
    elif dataset[i,0]=='IC2602':
        ax1.scatter(out[i,0],out[i,2],color='green',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='green',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='green',marker='4')
    elif dataset[i,0]=='IC4665':
        ax1.scatter(out[i,0],out[i,2],color='black',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='black',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='black',marker='.')
    elif dataset[i,0]=='NGC2232':
        ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='cyan',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='cyan',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0, fontsize='x-large')

In [ ]:
###Secondo grafico, con OC del training

import matplotlib.gridspec as gridspec
plt.close()
gs=gridspec.GridSpec(2,2)
gs.update(wspace=0.025, hspace=0.025)
fig=plt.figure()
ax1=fig.add_subplot(gs[0,0])
ax3=fig.add_subplot(gs[1,1])
ax2=fig.add_subplot(gs[0,1],sharex=ax3,sharey=ax1)
ax1.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=True, right=True, labelright=False)
ax2.tick_params(top=True, labeltop=True, bottom=True, labelbottom=False, left=True, labelleft=False, right=True, labelright=True)
ax3.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True, left=True, labelleft=False, right=True, labelright=True)
ax1.set_xlabel('n_(0)')
ax1.xaxis.set_label_position('top') 
ax1.set_ylabel('n_(2)')
ax2.set_xlabel('n_(1)')
ax2.xaxis.set_label_position('top')
ax2.set_ylabel('n_(2)')
ax2.yaxis.set_label_position('right')
ax3.set_xlabel('n_(1)')
ax3.set_ylabel('n_(0)')
ax3.yaxis.set_label_position('right')
ax1.scatter(out[340,0],out[340,2],color='red',marker='+',label='NGC2547')
ax1.scatter(out[130,0],out[130,2],color='blue',marker='1',label='NGC2516')
ax1.scatter(out[50,0],out[50,2],color='green',marker='4',label='NGC6253')
ax1.scatter(out[110,0],out[110,2],color='cyan',marker='x',label='NGC6633')
ax1.scatter(out[70,0],out[70,2],color='black',marker='.',label='M67')

for i in range(0,dataset.shape[0]):
    if dataset[i,0]=='NGC2547':
        ax1.scatter(out[i,0],out[i,2],color='red',marker='+')
        ax2.scatter(out[i,1],out[i,2],color='red',marker='+')
        ax3.scatter(out[i,1],out[i,0],color='red',marker='+')
    elif dataset[i,0]=='NGC2516':
        ax1.scatter(out[i,0],out[i,2],color='blue',marker='1')
        ax2.scatter(out[i,1],out[i,2],color='blue',marker='1')
        ax3.scatter(out[i,1],out[i,0],color='blue',marker='1')
    elif dataset[i,0]=='NGC6253':
        ax1.scatter(out[i,0],out[i,2],color='green',marker='4')
        ax2.scatter(out[i,1],out[i,2],color='green',marker='4')
        ax3.scatter(out[i,1],out[i,0],color='green',marker='4')
    elif dataset[i,0]=='NGC6633':
        ax1.scatter(out[i,0],out[i,2],color='black',marker='.')
        ax2.scatter(out[i,1],out[i,2],color='black',marker='.')
        ax3.scatter(out[i,1],out[i,0],color='black',marker='.')
    elif dataset[i,0]=='M67':
        ax1.scatter(out[i,0],out[i,2],color='cyan',marker='x')
        ax2.scatter(out[i,1],out[i,2],color='cyan',marker='x')
        ax3.scatter(out[i,1],out[i,0],color='cyan',marker='x')

fig.legend(bbox_to_anchor=(0.4,0.4), loc="upper right", borderaxespad=0, fontsize='x-large')

In [ ]:
###Prova plot 3d fatta per i nuovi OC

plt.close()
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
for i in range(0,dataset.shape[0]):
    if dataset[i,0]=='Pleiades':
        ax.scatter(out[i,0],out[i,1],out[i,2],color='red',marker='+')
    elif dataset[i,0]=='NGC2264':
        ax.scatter(out[i,0],out[i,1],out[i,2],color='blue',marker='1')
    elif dataset[i,0]=='IC2602':
        ax.scatter(out[i,0],out[i,1],out[i,2],color='green',marker='4')
    elif dataset[i,0]=='IC4665':
        ax.scatter(out[i,0],out[i,1],out[i,2],color='black',marker='.')
    elif dataset[i,0]=='NGC2232':
        ax.scatter(out[i,0],out[i,1],out[i,2],color='cyan',marker='x')